# This is the project in Optimization 1

In [ ]:
from functions import *
import numpy as np
import matplotlib.pyplot as plt
# from numpy.random import default_rng
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.datasets import load_iris

In [ ]:
np.random.seed(42)
w = np.array([1.,1.])
b = 1.

n = 1000
n_A = np.random.randint(0,n)
n_B = n-n_A
margin = 5.e-1
listA, listB = TestLinear(w,b,n_A,n_B,margin)

x = np.concatenate((np.array(listA),np.array(listB)))

y = np.concatenate((np.ones(n_A), -np.ones(n_B)))

plt.scatter(x[:, 0], x[:, 1], c=y, cmap='coolwarm', edgecolors='k')
plt.title("Data points")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")

plt.show()


In [ ]:

alpha0 = np.zeros(n_A+n_B)
tau = 1
niter = 1000
C = 3
G = pairwise_kernels(x, metric = kernal_linear)  

alpha = gradient_descent(alpha0, G, y, tau0=tau, niter=niter, C=C, tol=1e-7, projection=projection)

w, b = w_b(alpha, y,x ,  C=C)
plot_solution(x, y, w, b)


C = 45
G = pairwise_kernels(x, metric = kernal_linear)  

alpha = gradient_descent(alpha0, G, y, tau0=tau, niter=niter, C=C, tol=1e-7, projection=projection)

w, b = w_b(alpha, y,x ,  C=C)
plot_solution(x, y, w, b)

In [ ]:

alpha0 = np.zeros(n_A+n_B)
tau = 1
niter = 1000
C = 5
G = pairwise_kernels(x, metric = kernal_linear)


alpha = gradient_descent(alpha0, G, y, tau0=tau, niter=niter, C=C, tol=1e-7, projection=projection)
alphagrad, fks = gradient_descent_linesearch(alpha0, G, y, tau0=tau, niter=niter, C=C, tol=1e-7, project=projection)

w, b = w_b(alpha, y,x ,  C=C)
plt.title("Gradient descent without linesearch")
plot_solution(x, y, w, b)

w, b = w_b(alphagrad, y,x ,  C=C)
plt.title("Gradient descent with linesearch")
plot_solution(x, y, w, b)


plot_db(x, y, alpha, ker = kernal_linear, C=C)

In [ ]:
plot_db(x, y, alphagrad, ker = kernal_linear, C=C)

### Testing the different kernels

In [ ]:
niter = 1000
C = 5

G = pairwise_kernels(x, metric = kernal_gaussian, sigma=1)
alpha = gradient_descent(alpha0, G, y, tau, niter, C)
# w, b = w_b(alpha, y, x, C=C)
# plot_solution(x, y, w, b)
plot_db(x, y, alpha, ker = kernal_gaussian, C=C)    

G = pairwise_kernels(x, metric = kernal_inv_multiquadratic, sigma=1)
alpha = gradient_descent(alpha0, G, y, tau, niter, C)
# w, b = w_b(alpha, y, x, C=C)
# plot_solution(x, y,  w, b)
plot_db(x, y, alpha, ker = kernal_inv_multiquadratic, C=C)

G = pairwise_kernels(x, metric = kernal_laplacian, sigma=1)
alpha = gradient_descent(alpha0, G, y, tau, niter, C)
# w, b = w_b(alpha, y, x, C=C)
# plot_solution(x, y,  w, b)
plot_db(x, y, alpha, ker = kernal_laplacian, C=C)



In [ ]:

alpha0 = np.ones(n_A+n_B)
tau = 1
niter = 1000
C = 3
G = pairwise_kernels(x, metric = kernal_linear)


alpha_test1, fks = gradient_descent_linesearch(alpha0, G, y, tau, niter=niter, C=0.8, L=10, tol=1e-5, project=projection)
alpha_test2 = gradient_descent(alpha0, G, y, tau, niter=niter, C=0.8, tol=1e-3)


In [ ]:
alpha0 = np.ones(n_A+n_B)*0.5

alpha_test1, fks = gradient_descent_linesearch(alpha0, G, y, tau, niter=1000, C=0.8, L=10, tol=1e-5)
alpha_test2 = gradient_descent(alpha0, G, y, tau, niter=1000, C=0.8, tol=1e-3)

In [ ]:
# ker = kernal_linear
ker = kernal_gaussian
ker = kernal_inv_multiquadratic
# ker = kernal_laplacian
C = 1
G = pairwise_kernels(x, metric = ker)  

# np.random.seed(42)
alpha, f = gradient_descent_linesearch(alpha0, G, y, tau0=tau, niter=niter, C=C, tol=1e-7, project=projection)

plot_db(x, y, alpha, ker)

### Testing with random initial contitions

In [ ]:
alpha0 = np.random.rand((n_A+n_B))
ker = kernal_gaussian
G = pairwise_kernels(x, metric = ker)  

niter = 1000
C = 5
alpha_line, fk = gradient_descent_linesearch(alpha0, G, y, tau, niter, C = C, L=10, tol=1e-7)
alpha_grad = gradient_descent(alpha0, G, y, tau, niter, C=C, tol=1e-7)

plot_db(x, y, alpha_line, ker, C=C)
plot_db(x, y, alpha_grad, ker, C=C)


### Using gradient descent on the iris dataset

In [ ]:
iris = load_iris()
X = iris.data[:, :2][0:100]  # We'll use only the first two features for visualization
Y = iris.target[0:100]  
Y[Y == 0] = -1  # Convert to -1 and 1 for SVM
Y[Y == 1] = 1


plt.figure(figsize=(10, 5))
plt.scatter(X[:, 0], X[:, 1], c=Y, cmap=plt.cm.Set1, s=50, edgecolors='k')
plt.title('Iris Dataset (First Two Features)')
plt.xlabel('Sepal Length (cm)')
plt.ylabel('Sepal Width (cm)')
plt.show()

alpha0 = np.zeros(len(Y))
ker = kernal_gaussian
G = pairwise_kernels(X, metric=ker)
C= 100

alpha, f = gradient_descent_linesearch(alpha0, G, Y, tau0=tau, niter=niter, C=C, L=10, tol=1e-7, project=projection)

plot_db(X, Y, alpha, ker=ker, C=C)

In [ ]:
def projection_AL(vector, lower_bounds, upper_bounds):
    dimension = len(vector)
    proj_vector = np.array([])
    for k in range(0, dimension):
        if vector[k]<lower_bounds[k]:
            proj_vector = np.append(proj_vector, np.array([lower_bounds[k]]))
        elif vector[k] > upper_bounds[k]:
            proj_vector = np.append(proj_vector, np.array([upper_bounds[k]]))
        else:
            proj_vector = np.append(proj_vector, np.array([vector[k]]))
    return proj_vector

In [ ]:
def BCLM(wbxis_0, d, M, C, lambd_0, mu_0, tol_1, tol_2, maxiter): #Algoritme 17.4 i boka
    
    tol_1_k = 1/mu_0
    tol_2_k = 1/mu_0**(0.1)

    wbxis_k = wbxis_0
    lambd_k = lambd_0

    mu_k = mu_0

    for _ in range(0, maxiter):

        wbxis_k = 0 #solution to projected gradient descent with tolerance tol_1_k
        distance_k = 0 #wbxis_k - projection

        c_k = 0#constraints evaluated at wbxis_k
        c_k_norm = np.linalg.norm(c_k)

        if c_k_norm <= tol_2_k:
            
            if c_k_norm <= tol_2 and distance_k <= tol_1:
                return wbxis_k, lambd_k
            
            lambd_k = lambd_k - mu_k*c_k
            tol_1_k = tol_1_k/mu_k
            tol_2_k = tol_2_k/mu_k**(0.9)
        
        else:
            mu_k = 100*mu_k
            tol_1_k = tol_1_k/mu_k
            tol_2_k = tol_2_k/mu_k**(0.1)

In [ ]:
def AL(w, b, xi, s, lambd, mu, x, y, C): #kontroller at dette er rett
    AL = 0.5*np.linalg.norm(w)**2
    for i in range(0, len(xi)):
        indreprod = np.inner(w, x[i])
        AL = AL + C*xi[i] - lambd[i]*(y[i]*(indreprod + b) + xi[i] - s[i] - 1) + 0.5*mu*(y[i]*(indreprod + b) + xi[i] - s[i] - 1)**2
    return AL

#print(AL(np.array([1,1]),1, np.array([1,1]),np.array([1,1]), np.array([1,1]), 2, np.array([np.array([1,1]), np.array([1,1])]), np.array([1,1]), 1))

In [ ]:
def grad_AL(w, b, xi, s, lambd, mu, x, y, C): #Kontroller at dette er rett
    
    d = len(w)
    M = len(xi)
    
    grad_AL = np.array([])
    
    #Elements from w
    for k in range(0, d):
        grad_k = w[k]
        for i in range(0, M):
            indresum = 0
            for l in range(0, d):
                indresum = indresum + 2*x[i][k]*x[i][l]*w[l]
            grad_k = grad_k - lambd[i]*y[i]*x[i][k] + 0.5*mu*(y[i]**2 * indresum + 2*y[i]**2 * b * x[i][k] + 2*y[i]*xi[i]*x[i][k] - 2*y[i]*s[i]*x[i][k] -2*y[i]*x[i][k])
        grad_AL = np.append(grad_AL, np.array([grad_k]))
    
    #Elements from b
    grad_b = 0
    for i in range(0, M):
        grad_b = grad_b - lambd[i]*y[i] + 0.5*mu*(2*y[i]**2*b + 2*y[i]**2*np.inner(w, x[i]) + 2*y[i]*xi[i] - 2*y[i]*s[i] - 2*y[i])
    grad_AL = np.append(grad_AL, np.array([grad_b]))

    #Elements from xi
    for i in range(0, M):
        grad_xi = C - lambd[i] + 0.5*mu*(2*xi[i] + 2*y[i]*np.inner(w, x[i]) + 2*y[i]*b - 2*s[i] - 2)
        grad_AL = np.append(grad_AL, np.array([grad_xi]))

    #Elements from s
    for i in range(0, M):
        grad_s = -lambd[i] + 0.5*mu*(2*s[i] - 2*y[i]*np.inner(w, x[i]) - 2*y[i]*b - 2*xi[i] + 2)
        grad_AL = np.append(grad_AL, np.array([grad_s]))
    
    return grad_AL

#print(grad_AL(np.array([1,1]),1, np.array([1,1]),np.array([1,1]), np.array([1,1]), 2, np.array([np.array([1,1]), np.array([1,1])]), np.array([1,1]), 1))

In [ ]:
def analytic_linesearch(d_k, A, alpha):
    dot1 = np.dot(d_k, np.dot(A, d_k))
    dot2 = np.dot(d_k, np.dot(A, alpha))
    dot3 = np.dot(alpha, np.dot(A, d_k))
    dot4 = np.sum(d_k)
    theta = - (0.5*dot2 + 0.5 *dot3 - dot4)/dot1
    return theta

def linesearch():
    pass
